In [3]:
import sagemaker
import boto3
import os

from sagemaker.predictor import Predictor
from sagemaker.pytorch import PyTorchModel

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

class ImagePredictor(Predictor):
  def __init__(self, endpoint_name, sagemaker_session):
    super(ImagePredictor, self).__init__(
      endpoint_name,
      sagemaker_session=sagemaker_session,
      serializer=sagemaker.serializers.IdentitySerializer("image/jpeg"),
      deserializer=sagemaker.deserializers.JSONDeserializer())
    
pytorch_model = PyTorchModel(model_data="s3://amazonbin/models/model_20220530.pth.tar.gz",
                             role=role,
                             entry_point='inference.py',
                             py_version='py3',
                             framework_version='1.8.0',
                             predictor_cls=ImagePredictor)

predictor = pytorch_model.deploy(initial_instance_count=1, instance_type='ml.m5.large')

------!

In [4]:
from PIL import Image
import io
buf = io.BytesIO()
Image.open("00024.jpg").save(buf, format="JPEG")

response = predictor.predict(buf.getvalue())

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-inference-2022-05-31-04-54-30-605 in account 170530745045 for more information.

In [5]:
predictor.delete_endpoint()